In [8]:
%%capture
%pip install -r ./requirements.txt

In [2]:
import numpy as np
print(np.__version__)

1.26.4


In [116]:
import os
import re
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as MetaNames
from langgraph.graph import StateGraph, START, END, MessagesState
from typing import TypedDict

In [74]:
class StateSchema:
    def __init__(self, input: str = "", agent: str = "", output: str = ""):
        self.input = input
        self.agent = agent
        self.output = output

In [75]:
load_dotenv()
model_id = "ibm/granite-34b-code-instruct" 
credentials = {
    "apikey": os.getenv("WATSONX_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com"
}

model = Model(
    model_id=model_id,
    credentials=credentials,
    project_id=os.getenv("WATSONX_PROJECT_ID")
)

In [145]:
def modelo_orquestador(state: StateSchema):
    # print('Paso por orquestador')
    prompt = f"""
    You are an AI orchestrator agent, and your job is to determine which agent to forward the input to. You will have two possible agents to pass the input to:
    
    - Translator: This agent will be responsible for translating the text into Spanish if necessary.
    - General: When you don't see any specific agent to handle the task, forward the input to this one.
    
    The way to choose the agent is simple: just repeat the input given to you but with a word in front of the input depending on which agent you should forward it to:
    - "translator" when it goes to the translator agent
    - "general" for the general agent.

    Input:{state.input}\n\nAssistant:
    """
    parameters = {
		MetaNames.DECODING_METHOD: "greedy",
		MetaNames.MAX_NEW_TOKENS: 100,
		MetaNames.MIN_NEW_TOKENS: 1,
		MetaNames.STOP_SEQUENCES: ["Input:", "\n\n"]
	}

    response = model.generate(prompt, parameters)

    generated_text = response['results'][0]['generated_text'].strip()

    if "translator" in generated_text.lower():
        # print('Voy a translator')
        state.agent = "translator"
        patron = r"^translator:\s*"
        generated_text = re.sub(patron, "", generated_text)
    else:
        # print('Voy a general')
        state.agent = "general"
        patron = r"^general:\s*"
        generated_text = re.sub(patron, "", generated_text)
    
    # print(f'Texto del orquestador: {generated_text}')
    state.input = generated_text
    return state

In [146]:
def modelo_traductor(state: StateSchema):
    # print('Paso por traductor')
    # prompt = f"""
    # You are an AI translator agent, and your job is to translate into Spanish all the input you receive.
    # The first word of each input you receive will be "translator," and you should disregard that word, translating everything in the input starting from that word.

    # Input:{state.input}\n\nAssistant:
    # """
    
    prompt = f"""
    Translate to Spanish
    English text: '{state.input}',
    Spanish text:
    """

    parameters = {
		MetaNames.DECODING_METHOD: "greedy",
		MetaNames.MAX_NEW_TOKENS: 100,
		MetaNames.MIN_NEW_TOKENS: 1,
		MetaNames.STOP_SEQUENCES: ["Input:", "\n\n"]
	}

    response = model.generate(prompt, parameters)

    generated_text = response['results'][0]['generated_text'].strip()
    # print(f'Texto generado en el traductor: {generated_text}')
    state.output = generated_text
    return state

In [150]:
def modelo_general(state: StateSchema):
    # print('Paso por general')
    prompt = f"""
    You are a general AI agent.
    The first word of each input you receive will be "general," and you should disregard that word, providing a response for the input starting from that word.

    Input:{state.input}\n\nAssistant:
    """

    parameters = {
		MetaNames.DECODING_METHOD: "greedy",
		MetaNames.MAX_NEW_TOKENS: 100,
		MetaNames.MIN_NEW_TOKENS: 1,
		MetaNames.STOP_SEQUENCES: ["Input:", "\n\n"]
	}

    response = model.generate(prompt, parameters)

    generated_text = response['results'][0]['generated_text'].strip()
    
    state.output = generated_text
    return state

In [151]:
def decidir_transicion(state):
    print(f'State agent: {state.agent}')
    return "traductor" if state.agent == "translator" else "general"

In [ ]:
grafo = StateGraph(state_schema=StateSchema)


grafo.add_node("clasificar_intencion", modelo_orquestador)
grafo.add_node("traductor", modelo_traductor)
grafo.add_node("general", modelo_general)

grafo.add_edge(START, "clasificar_intencion")
grafo.add_conditional_edges("clasificar_intencion", decidir_transicion)
grafo.add_edge("traductor", END)
grafo.add_edge("general", END)

agente = grafo.compile()


estado = agente.invoke(StateSchema('Hi, can you translate this sentence into Spanish? Is there cold water?'))
print(estado.output)
estado = agente.invoke(StateSchema('What is seven plus two'))
print(estado.output)

State agent: translator
'¿Hay agua fría?'
State agent: general
9
